In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifiersource venv/bin/activate
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import joblib


# Estrazione Sintomi da Testo Libero (NLP)

In questo notebook implementiamo una pipeline NLP per estrarre automaticamente i sintomi da descrizioni testuali, tipiche dei pazienti in contesti reali. L'obiettivo è trasformare un testo in input in una rappresentazione strutturata compatibile con i modelli di classificazione delle malattie.


In [ ]:
# Per Colab (decommentare)
# !pip install spacy
# !python -m spacy download en_core_web_sm


In [ ]:
import spacy

# Carica modello spaCy
nlp = spacy.load("en_core_web_sm")

# Sintomi noti (semplificati)
symptom_keywords = {
    "fever": "fever",
    "headache": "headache",
    "cough": "cough",
    "sore throat": "sore_throat",
    "fatigue": "fatigue",
    "nausea": "nausea",
    "diarrhea": "diarrhea",
    "vomiting": "vomiting",
    "chills": "chills",
    "shortness of breath": "shortness_of_breath",
    "muscle pain": "muscle_pain"
}


In [ ]:
def extract_symptoms(text, symptom_dict):
    doc = nlp(text.lower())
    found = set()
    for phrase, label in symptom_dict.items():
        if phrase in doc.text:
            found.add(label)
    return found


In [ ]:
text_input = "Da tre giorni ho la febbre, tosse secca e dolore muscolare. Sono anche molto stanco."
extracted = extract_symptoms(text_input, symptom_keywords)

# Mostra risultato
print("Sintomi trovati:", extracted)


In [ ]:
def symptom_vector(symptoms, all_symptoms):
    return [1 if s in symptoms else 0 for s in all_symptoms]

feature_vector = symptom_vector(extracted, symptom_keywords.values())
print("Vettore binario:", feature_vector)


In [14]:
# Carica dataset (adatta il path se serve)
df = pd.read_csv("/content/drive/MyDrive/BigData/dataset_weighted.csv")  # Cambia path se non usi Colab

# Seleziona colonne sintomi
symptom_cols = [col for col in df.columns if "Symptom_" in col]
X = df[symptom_cols]
y = df["Disease"]


In [16]:
# Suddivisione train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Addestramento modello
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Valutazione
y_pred = model.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print("Accuracy su test set:", acc)

# Salvataggio in .pkl
joblib.dump(model, "/content/drive/MyDrive/BigData/modello_rf.pkl")
print("Modello salvato in 'modello_rf.pkl'")


Accuracy su test set: 0.994579945799458
Modello salvato in 'modello_rf.pkl'


# predict

In [17]:
import spacy
from deep_translator import GoogleTranslator
import joblib
import numpy as np

# Carica modello NLP
nlp = spacy.load("en_core_web_sm")

# Carica modello di classificazione
model = joblib.load("/content/drive/MyDrive/BigData/modello_rf.pkl")  # Assicurati che il file .pkl sia nella stessa directory


In [18]:
# Mappa di sintomi (sinonimi e varianti incluse)
symptom_keywords = {
    "fever": "fever", "high fever": "fever", "febbre": "fever",
    "headache": "headache", "mal di testa": "headache",
    "cough": "cough", "dry cough": "cough", "tosse": "cough",
    "sore throat": "sore_throat", "mal di gola": "sore_throat",
    "fatigue": "fatigue", "tired": "fatigue", "stanco": "fatigue",
    "nausea": "nausea",
    "vomiting": "vomiting", "vomito": "vomiting",
    "diarrhea": "diarrhea", "diarrea": "diarrhea",
    "chills": "chills", "brividi": "chills",
    "shortness of breath": "shortness_of_breath", "fiato corto": "shortness_of_breath",
    "muscle pain": "muscle_pain", "dolore muscolare": "muscle_pain"
}


In [21]:
def extract_symptoms(text, symptom_dict):
    doc = nlp(text.lower())
    found = set()
    for phrase, label in symptom_dict.items():
        if phrase in doc.text:
            found.add(label)
    return found

def symptom_vector(symptoms, training_columns, symptom_dict):
    # Create a mapping from symptom label to symptom_column index
    symptom_to_col = {label: i for i, col in enumerate(training_columns) for phrase, label in symptom_dict.items() if label in col}

    # Initialize a vector of zeros with the same length as training_columns
    vector = [0] * len(training_columns)

    # Set the corresponding index to 1 if the symptom is found
    for s in symptoms:
        if s in symptom_to_col:
            vector[symptom_to_col[s]] = 1
    return vector

In [22]:
# Inserisci frase in italiano
input_text_it = "Ho mal di gola e febbre da due giorni."

# Traduzione automatica
input_text_en = GoogleTranslator(source='auto', target='en').translate(input_text_it)
print("Frase tradotta:", input_text_en)

# Estrazione sintomi
extracted = extract_symptoms(input_text_en, symptom_keywords)
print("Sintomi estratti:", extracted)

# Vettore sintomi
# Use the columns from the training data X_train to ensure correct feature order and number
vector = symptom_vector(extracted, X_train.columns, symptom_keywords)
print("Vettore binario:", vector)

# Predizione
prediction = model.predict([vector])
print(" Malattia Predetta:", prediction[0])

Frase tradotta: I have been on the sore throat and fever for two days.
Sintomi estratti: {'sore_throat', 'fever'}
Vettore binario: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
 Malattia Predetta: Acne


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
